In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rlway.pyosrd import OSRD

In [3]:
OSRD.use_cases

['cvg_dvg', 'point_switch', 'station_capacity2']

In [218]:
use_case='point_switch'
sim = OSRD(use_case=use_case, dir=use_case)

[16:14:53,329] [INFO]  [StandaloneSimulationCommand] Loading infra: point_switch/infra.json
[16:14:53,361] [INFO]              [FullInfra] parsing infra
[16:14:53,421] [INFO]              [FullInfra] adaptation to kotlin
[16:14:53,452] [INFO]              [FullInfra] loading signals
[16:14:53,457] [INFO]              [FullInfra] building blocks
[16:14:53,462] [INFO]  [StandaloneSimulationCommand] Loading input: point_switch/simulation.json
[16:14:53,473] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.28
[16:14:53,520] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.29
[16:14:53,525] [INFO]  [StandaloneSimulationCommand] All simulations completed
[16:14:53,525] [INFO]  [StandaloneSimulationCommand] Saving results at: point_switch/results.json


In [219]:
sim.route_lengths

{'rt.buffer_stop.0->D0': 9820.0,
 'rt.D0->buffer_stop.1': 10180.0,
 'rt.D0->buffer_stop.2': 10180.0,
 'rt.buffer_stop.1->D1': 9820.0,
 'rt.D1->buffer_stop.0': 10180.0,
 'rt.buffer_stop.2->D2': 9820.0,
 'rt.D2->buffer_stop.0': 10180.0}

In [220]:
from rlway.schedule_from_osrd import schedule_from_osrd

In [221]:
schedule_from_osrd(sim)

KeyError: 0